# Homework 7 Question 3: ABC Investments.
ABC Inc. is considering several investment options. Each option has
a minimum investment required as well as a maximum investment allowed. These restrictions, along
with the expected return are summarized in the following table (figures are in millions of dollars): 

Option | Minimum investment | Maximum investment | Expected return (%) |
--- | --- | --- | --- |
1 | 3 | 27 | 13
2 | 2 | 12 | 9
3 | 9 | 35 | 17
4 | 5 | 15 | 10
5 | 12| 46 | 22
6 | 4 | 18 | 12

Because of the high-risk nature of Option 5, company policy requires that the total amount invested
in Option 5 be no more that the combined amount invested in Options 2, 4 and 6. In addition, if
an investment is made in Option 3, it is required that at least a minimum investment be made in
Option 6. ABC has $80 million to invest and obviously wants to maximize its total expected return
on investment. Which options should ABC invest in, and how much should be invested?

## Problem Data 

In [1]:
options = [:one, :two, :three, :four, :five, :six]
minInv = Dict(zip(options, [3, 2, 9, 5, 12, 4]))
maxInv = Dict(zip(options, [27, 12, 35, 15, 46, 18]))
expReturn = Dict(zip(options, [13, 9, 17, 10, 22, 12]));

## Problem Model 

In [2]:
using JuMP, Cbc

m = Model(solver=CbcSolver())

# Boolean decision variable to correspond to stocks invested in
@variable(m, investedIn[options], Bin)
@variable(m, investment[options]) # Investement made in each stock

# Min investment constraint
@constraint(m, minConstr[o in options], investment[o] >= investedIn[o]*minInv[o])
# Max investment constraint
@constraint(m, maxConstr[o in options], investment[o] <= investedIn[o]*maxInv[o])

# Constraint for 5 equal to 2+4+6
@constraint(m, investment[:five] <= sum(investment[o] for o in [:two, :four, :six]))
# Constraint for 6 to be compulsory if invested in three
@constraint(m, investment[:six] >= investedIn[:three]*minInv[:six])

# Total investment available constraint
@constraint(m, sum(investment[o] for o in options) <= 80)

# Objective to maximize profit / return.
@objective(m, Max, sum(investment[o]*(expReturn[o]/100.) for o in options))
m

Maximization problem with:
 * 15 linear constraints
 * 12 variables: 6 binary
Solver is CbcMathProg

In [3]:
status = solve(m)
println("Status: ", status)
println("Investments ", getvalue(investment))
println("Invested In ", getvalue(investedIn))
println("Expected Return ", getobjectivevalue(m))
println("Expected Return (%) ", getobjectivevalue(m)*100/sum(getvalue(investment)))

Status: Optimal
Investments investment: 1 dimensions:
[  one] = 0.0
[  two] = 0.0
[three] = 34.99999999999999
[ four] = 5.000000000000001
[ five] = 22.500000000000004
[  six] = 17.5

Invested In investedIn: 1 dimensions:
[  one] = 0.0
[  two] = 0.0
[three] = 1.0
[ four] = 1.0
[ five] = 1.0
[  six] = 1.0

Expected Return 13.500000000000004
Expected Return (%) 16.875000000000007
